In [1]:
# All  import statements needed for the notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import *

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data = pd.read_csv('/content/drive/MyDrive/balancedhalf_data.csv')

data.drop(data.columns[0] , inplace=True , axis=1)

unknown = ['SSH-Patator','DoS slowloris','DoS Slowhttptest','Bot','Infiltration','Heartbleed']
att = data.loc[(data['Label'].isin(unknown))]

data.drop(att.index,axis=0 , inplace=True, errors='ignore')

In [5]:
## PreProcessing

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

# using standard scaler for normalizing
std_scaler = MinMaxScaler()
def normalization(df,att,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    x = np.array(att[i])
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
    #To use the same scaler which was used in preprocessing the train data 
    att[i] = std_scaler.transform(x.reshape(len(x),1))
  return df,att
# calling the normalization() function
data , att = normalization(data.copy(),att.copy(),numeric_col)

data.shape , att.shape

att.Label = 'unknown'

X = att.drop('Label' , axis=1)
X = X.to_numpy().reshape(-1, 83,1)

y = att.Label

X_train = data.drop('Label' , axis=1)
X_train = X_train.to_numpy().reshape(-1, 83,1)

y_train = data.Label

In [6]:
# **Load Model and Predict**
MODEL = '/content/drive/MyDrive/models/model_split_softmax_cnn_model.hdf5'
from tensorflow import keras
model = keras.models.load_model(MODEL)

In [7]:
y_train.value_counts()

BENIGN           555466
DoS Hulk         231073
PortScan         158930
DDoS             128027
DoS GoldenEye     10293
FTP-Patator        7938
Name: Label, dtype: int64

In [8]:
Y_train_predicted = model.predict(X_train)

34117/34117 [==============================] - 82s 2ms/step


In [9]:
LABELS = ['BENIGN','DDoS','DoS GoldenEye','DoS Hulk','FTP-Patator','PortScan']

In [10]:
y_prediction_index = np.argmax(Y_train_predicted, axis = 1)

In [11]:
df = pd.DataFrame(data=Y_train_predicted,columns=LABELS)

In [12]:
labeltoindex = {
    'BENIGN':0,'DDoS':1,'DoS Hulk':3,'DoS GoldenEye':2,'PortScan':5,'FTP-Patator':4
}

In [13]:
np.vectorize(labeltoindex.get)(y_train)

array([0, 0, 0, ..., 4, 4, 4])

In [14]:
TRUELY_PREDICTED = np.vectorize(labeltoindex.get)(y_train) == y_prediction_index

In [15]:
TRUELY_PREDICTED

array([ True,  True,  True, ...,  True,  True,  True])

In [16]:
df.head()

,BENIGN,DDoS,DoS GoldenEye,DoS Hulk,FTP-Patator,PortScan
0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


In [17]:
pd.Series(TRUELY_PREDICTED).value_counts()

True     1091076
False        651
dtype: int64

In [18]:
df['INDEX_WIN'] = np.where( TRUELY_PREDICTED == True , y_prediction_index, '')

In [19]:
df.shape

(1091727, 7)

In [20]:
df = df[df['INDEX_WIN'] != '']

In [21]:
df.shape

(1091076, 7)

In [22]:
df['INDEX_WIN'].value_counts()

0    554933
3    231071
5    158920
1    128024
2     10208
4      7920
Name: INDEX_WIN, dtype: int64

In [32]:
df.dtypes

BENIGN           float32
DDoS             float32
DoS GoldenEye    float32
DoS Hulk         float32
FTP-Patator      float32
PortScan         float32
INDEX_WIN         object
dtype: object

In [37]:
df.INDEX_WIN = df.INDEX_WIN.astype('int')

In [38]:
df.dtypes

BENIGN           float32
DDoS             float32
DoS GoldenEye    float32
DoS Hulk         float32
FTP-Patator      float32
PortScan         float32
INDEX_WIN          int64
dtype: object

In [39]:
threshold_values = {}
def to_do(df):
  ind = df['INDEX_WIN'].to_numpy()[0]
  LABELS = ['BENIGN','DDoS','DoS GoldenEye','DoS Hulk','FTP-Patator','PortScan']  
  label = LABELS[ind] 
  import numpy as np
  threshold =  np.amin(df[label])
  threshold_values[label] = threshold
  return threshold

In [40]:
df.groupby(['INDEX_WIN'], group_keys=True).apply(to_do)

INDEX_WIN
0    0.349928
1    1.000000
2    0.974300
3    0.700373
4    1.000000
5    0.910532
dtype: float32

In [41]:
threshold_values

{'BENIGN': 0.3499285,
 'DDoS': 1.0,
 'DoS GoldenEye': 0.9743001,
 'DoS Hulk': 0.7003728,
 'FTP-Patator': 0.9999999,
 'PortScan': 0.9105322}